In [1]:
from pydrive2.auth import GoogleAuth, ServiceAccountCredentials
from pydrive2.drive import GoogleDrive

# Load service account and credentials
gauth = GoogleAuth()
gauth.LoadCredentialsFile("trl-doc-uploader-ca4496d60409.json")

drive = GoogleDrive(gauth)

# Define folder ID
FOLDER_ID = "1-K5WSgmRiCJD6NI4SQ8VFKLcn6yw-aou"

# Upload a file
file_metadata = {
    "title": "Uploaded_file.txt",
    "parents": [{"id": FOLDER_ID}]
}

file = drive.CreateFile(file_metadata)
file.SetContentString("Hello, this is an automated upload!")
file.Upload()
print("Success")

KeyError: '_module'

In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

# Define scope
scopes = ["https://www.googleapis.com/auth/drive"]

# Load credentials from JSON file
creds = ServiceAccountCredentials.from_json_keyfile_name("trl-doc-uploader-ca4496d60409.json", scopes)

# Authenticate
gauth = GoogleAuth()
gauth.credentials = creds
drive = GoogleDrive(gauth)

# Test authentication
print("Authenticated successfully!")


Authenticated successfully!


In [7]:
# List all folders in the Service Account's Drive
query = "mimeType='application/vnd.google-apps.folder' and trashed=false"
folder_list = drive.ListFile({'q': query}).GetList()

print("Folders found:")
for folder in folder_list:
    print(f"- {folder['title']} (ID: {folder['id']})")


Folders found:
- Prueba_Nuevo_Folder (ID: 18ho9dmVxwWwN7svyknFXDAtxw3hHY5G2)
- TRL (ID: 1-K5WSgmRiCJD6NI4SQ8VFKLcn6yw-aou)


In [8]:
trl_folder_id = "1-K5WSgmRiCJD6NI4SQ8VFKLcn6yw-aou"  # Your TRL folder ID

file_metadata = {
    'title': 'test_file.txt',
    'parents': [{'id': trl_folder_id}]
}

file = drive.CreateFile(file_metadata)
file.SetContentString("This is a test file inside the TRL folder.")
file.Upload()
print("File uploaded successfully to TRL folder!")


File uploaded successfully to TRL folder!


In [9]:
query = f"'{trl_folder_id}' in parents and trashed=false"
file_list = drive.ListFile({'q': query}).GetList()

print("Files inside TRL folder:")
for file in file_list:
    print(f"- {file['title']} (ID: {file['id']})")

Files inside TRL folder:
- test_file.txt (ID: 1sPCJKyyRGghUtQ3CTMEK3KnxZGwgBuO3)


In [6]:
def get_or_create_folder(drive, folder_name):
    # Search for folder
    query = f"title = '{folder_name}' and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    folder_list = drive.ListFile({'q': query}).GetList()

    if folder_list:
        print(f"Folder '{folder_name}' found.")
        return folder_list[0]['id']  # Return existing folder ID
    else:
        print(f"Folder '{folder_name}' not found. Creating it...")
        folder_metadata = {
            'title': folder_name,
            'mimeType': 'application/vnd.google-apps.folder'
        }
        folder = drive.CreateFile(folder_metadata)
        folder.Upload()
        print(f"Folder '{folder_name}' created successfully!")
        return folder['id']  # Return new folder ID
    
folder_id = get_or_create_folder(drive, "Prueba_Nuevo_Folder")



Folder 'Prueba_Nuevo_Folder' found.


In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from oauth2client.service_account import ServiceAccountCredentials

class GoogleDriveFolder:
    def __init__(self, credentials_file, folder_id=None):
        """
        Initializes the GoogleDriveFolder instance and authenticates.
        :param credentials_file: Path to the service account JSON file.
        :param folder_id: Folder ID to work with (None if you want to work with root).
        """
        self.credentials_file = credentials_file
        self.folder_id = folder_id
        self.drive = self.authenticate_google_drive()

    def authenticate_google_drive(self):
        """
        Authenticates using the Service Account and returns the GoogleDrive instance.
        """
        scopes = ["https://www.googleapis.com/auth/drive"]
        creds = ServiceAccountCredentials.from_json_keyfile_name(self.credentials_file, scopes)
        gauth = GoogleAuth()
        gauth.credentials = creds
        drive = GoogleDrive(gauth)
        
        print("Authenticated successfully!")
        return drive

    def get_folder_files(self):
        """
        Lists the files in the root folder or the specified subfolder.
        :return: A list of files.
        """
        query = f"'{self.folder_id}' in parents and trashed=false" if self.folder_id else "trashed=false"
        file_list = self.drive.ListFile({'q': query}).GetList()
        
        print("Files found:")
        for file in file_list:
            print(f"- {file['title']} (ID: {file['id']})")
        return file_list

    def create_folder(self, folder_name):
        """
        Creates a new folder inside the specified folder or root.
        :param folder_name: The name of the folder to create.
        :return: The ID of the created folder.
        """
        folder_metadata = {
            'title': folder_name,
            'mimeType': 'application/vnd.google-apps.folder',
            'parents': [{'id': self.folder_id}] if self.folder_id else []
        }
        
        folder = self.drive.CreateFile(folder_metadata)
        folder.Upload()
        
        print(f"Folder '{folder_name}' created successfully!")
        return folder['id']

    def upload_file(self, file_path, user_name, file_name=None):
        """
        Uploads a file to the specified folder (or root if no folder ID).
        :param file_path: The path to the file to upload.
        :param user_name: The name of the user to prefix the file with.
        :param file_name: The custom file name (optional, if None uses the original).
        :return: The uploaded file's ID.
        """
        if not file_name:
            file_name = file_path.split('/')[-1]

        file_metadata = {
            'title': f"{user_name}_{file_name}",
            'parents': [{'id': self.folder_id}] if self.folder_id else []
        }
        
        file = self.drive.CreateFile(file_metadata)
        file.SetContentFile(file_path)
        file.Upload()
        
        print(f"File '{file_name}' uploaded successfully!")
        return file['id']

# Example Usage:
# Assuming service_account.json is your credentials file and the folder ID is known

# Create an instance for the TRL folder
folder = GoogleDriveFolder(credentials_file="service_account.json", folder_id="1-K5WSgmRiCJD6NI4SQ8VFKLcn6yw-aou")

# 1. List files in the folder
folder.get_folder_files()

# 2. Create a new folder
new_folder_id = folder.create_folder("New_SubFolder")

# 3. Upload a PDF file
file_id = folder.upload_file("path_to_your_file.pdf", user_name="JohnDoe", file_name="test_upload.pdf")
